In [15]:
import numpy as np
import torch
from datetime import datetime
import glob
import os
import math
import pandas as pd

In [16]:
def index2regret(indices, y):
    indices = np.array(indices, dtype=int)
    regret = torch.zeros((indices.shape[0], indices.shape[1]))
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            regret[i, j] = 1 - y[indices[i, :j+1]].max()
    return regret

In [17]:
def load_tensors(subdir):
    tensors = sorted(glob.glob(subdir + '/*indices*.pt'))
        
    tests = {}
    print('Models not included/not over budget yet:\n')

    for tensor in tensors:
        num = int(tensor.split('indices')[0].split('_')[-1])

        if '.pt' in tensor and 'state_dict' not in tensor:
            first = False
            if "Random" not in os.path.basename(tensor):
                nm = os.path.basename(tensor).split('_')[0] + '_' + os.path.basename(tensor).split('_')[1]
            else:     
                nm = os.path.basename(tensor).split('_')[0]

            t = torch.load(tensor).cpu().detach()
            t = torch.reshape(t, (1, -1))

            lim = 480
            if t.size(-1) < lim:
                if 'indices.pt' in tensor:
                    print(tensor.split('/')[-1])
                continue
            if nm in tests.keys():
                d = tests[nm]
            else:
                d = {}
                tests[nm] = d

            dtype = os.path.basename(tensor).split('_')[-1].split('.')[0]
            dtype = ''.join([i for i in dtype if not i.isdigit()])
            if dtype in d.keys():
                arr = d[dtype]
            else:
                arr = t
                d[dtype] = arr
                first = True
            if first:
                pass
            elif t.size(-1) < arr.size(-1):
                arr = torch.cat((arr[:,:t.size(-1)], t), 0)
            else:
                arr = torch.cat((arr, t[:,:arr.size(-1)]), 0)
            d[dtype] = arr
            tests[nm] = d

    print('\nModels included:\n')
    batch = {}
    budget, total= math.inf, math.inf
    for key in tests.keys():
        print(key)
        num_runs = -1
        for dtype in tests[key].keys():
            t = tests[key][dtype]
            t = index2regret(t, y)
            
            if 'indices' == dtype and t.size(-1) != 0:
                if t.size(-1) < budget:
                    budget = t.size(-1)
                num_runs = t.size(0)
            elif 'y' in dtype and t.size(-1) < total and t.size(-1) != 0:
                total = t.size(-1)
            
            sd, mean = torch.std_mean(t, 0)
            sem = sd / (t.size(0)**.5)
            if dtype in batch.keys():
                d = batch[dtype]
            else:
                d = {}
                batch[dtype] = d

            d[key] = (mean, sem)

            batch[dtype] = d
        print("Runs: {}".format(num_runs))
    print(batch.keys())
    print('Budget: {}'.format(budget))
    print('Total queries (incl. init): {}'.format(budget))

    return batch, budget

In [27]:
def tabulate_regret(df, tests, budget, subdir, randregret=None):
    names = []
    queries = np.arange(budget) + 1
    
    for name in sorted(tests.keys()):
        names.append(name)

        mean, sem = tests[name]
        mean = 1 - mean
        if mean.size(0) < budget:
            names.pop()
            continue
        if mean.size(0) > budget:
            mean = mean[:budget]
            sem = sem[:budget]

        protein = subdir.split('/')[-1].split('_')[0]
        encoding = subdir.split('/')[-1].split('_')[1]

        if 'Random' in name:
            encoding = 'Random'
            model = 'Random'
            acquisition = 'Random'
        else:
            names = name.split('-')
            model = names[0]
            acquisition = names[-2]

        for timestep, single_mean, single_std in zip(queries, np.array(mean), np.array(sem)):
            df.loc[len(df.index)] = [protein, encoding, model, acquisition, timestep, single_mean, single_std]
    return df

In [28]:
df = pd.DataFrame(columns=['Protein', 'Encoding', 'Model', 'Acquisition', 'Timestep', 'Mean', 'Std'])

folders = ['GB1_AA', 'GB1_onehot', 'GB1_georgiev', 'GB1_ESM2', 'TrpB_AA', 'TrpB_onehot', 'TrpB_georgiev', 'TrpB_ESM2']

for protein in ['GB1', 'TrpB']:
    for encoding in ['AA', 'georgiev', 'onehot', 'ESM2']:
        subdir =  '../results/5x96_simulations/' + protein + '/' + encoding
        fitness_df = pd.read_csv('../data/' + protein + '/fitness.csv')
        y = fitness_df['fitness'].values
        y = y/y.max()
       
        batch, budget = load_tensors(subdir)
        df = tabulate_regret(df, batch['indices'], budget, subdir, randregret=None)

Models not included/not over budget yet:


Models included:

BOOSTING_ENSEMBLE-DO-0-RBF-GREEDY-[16, 1]
Runs: 70
BOOSTING_ENSEMBLE-DO-0-RBF-TS-[16, 1]
Runs: 70
BOOSTING_ENSEMBLE-DO-0-RBF-UCB-[16, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-GREEDY-[8, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-TS-[8, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-UCB-[8, 1]
Runs: 70
DNN_ENSEMBLE-DO-0-RBF-GREEDY-[8, 1]
Runs: 70
DNN_ENSEMBLE-DO-0-RBF-TS-[8, 1]
Runs: 70
DNN_ENSEMBLE-DO-0-RBF-UCB-[8, 1]
Runs: 70
GP_BOTORCH-DO-0-RBF-GREEDY-[16, 1]
Runs: 70
GP_BOTORCH-DO-0-RBF-TS-[16, 1]
Runs: 70
GP_BOTORCH-DO-0-RBF-UCB-[16, 1]
Runs: 70
dict_keys(['indices'])
Budget: 480
Total queries (incl. init): 480
Models not included/not over budget yet:


Models included:

BOOSTING_ENSEMBLE-DO-0-RBF-GREEDY-[80, 1]
Runs: 70
BOOSTING_ENSEMBLE-DO-0-RBF-TS-[80, 1]
Runs: 70
BOOSTING_ENSEMBLE-DO-0-RBF-UCB-[80, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-GREEDY-[30, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-TS-[30, 1]
Runs: 70
DKL_BOTORCH-DO-0-RBF-UCB-[30, 1]
Runs: 70
DNN_ENSEMBLE-

In [29]:
df = df.drop_duplicates(subset=['Protein', 'Encoding', 'Model', 'Acquisition', 'Timestep'], keep='first')
df['Model'] = df['Model'].replace('BOOSTING_ENSEMBLE', 'Boosting Ensemble')
df['Model'] = df['Model'].replace('GP_BOTORCH', 'GP')
df['Model'] = df['Model'].replace('DNN_ENSEMBLE', 'DNN Ensemble')
df['Model'] = df['Model'].replace('DKL_BOTORCH', 'DKL')
df['Acquisition'] = df['Acquisition'].replace('Random', 'GREEDY')
df

,Protein,Encoding,Model,Acquisition,Timestep,Mean,Std
0,GB1,AA,Boosting Ensemble,GREEDY,1,0.010112,0.004871
1,GB1,AA,Boosting Ensemble,GREEDY,2,0.021386,0.007114
2,GB1,AA,Boosting Ensemble,GREEDY,3,0.043239,0.011129
3,GB1,AA,Boosting Ensemble,GREEDY,4,0.046141,0.011136
4,GB1,AA,Boosting Ensemble,GREEDY,5,0.050109,0.011454
...,...,...,...,...,...,...,...
46075,TrpB,ESM2,GP,UCB,476,0.643956,0.005861
46076,TrpB,ESM2,GP,UCB,477,0.643956,0.005861
46077,TrpB,ESM2,GP,UCB,478,0.643956,0.005861
46078,TrpB,ESM2,GP,UCB,479,0.643956,0.005861


In [30]:
df.to_csv('all_results.csv', index=False)